In [ ]:
import pandas as pd
from warcio.archiveiterator import ArchiveIterator
from collections import Counter
import re
from tqdm import tqdm
import os
from urllib.parse import urlparse
import json

In [3]:
df = pd.read_csv('/project/0_cleaning/output_cleaning/ecom-full.csv')
df.head()

,ecommerce,wapp_folder,wapps
0,CC-MAIN-20171119004302-20171119024302-00601.wa...,wapp-00,['PrestaShop']
1,CC-MAIN-20171120032907-20171120052907-00562.wa...,wapp-00,['OpenCart']
2,CC-MAIN-20171124180349-20171124200349-00561.wa...,wapp-00,['Magento']
3,CC-MAIN-20171123050243-20171123070243-00112.wa...,wapp-00,['OpenCart']
4,CC-MAIN-20171117223659-20171118003659-00491.wa...,wapp-00,['Magento']


# Create WARC summarising dataframe

In [55]:
table = []

for entry in tqdm(df['ecommerce']):
    
    file_path = '/mnt/bigdata/s3/'+ entry
    stream = open(file_path, 'rb')
    for record in ArchiveIterator(stream):
        if record.rec_type == 'response':
            url = record.rec_headers.get_header('WARC-Target-URI')
            ip = record.rec_headers.get_header('WARC-IP-Address')
            content = record.http_headers.get_header('Content-Type')
    stream.close()    

    table.append({
        'file': entry,
        'ip': ip,
        'url': url,
        'content-type': content,
        'wapps': df['wapps'][df['ecommerce'] == entry].tolist()[0]
    })

df_warc = pd.DataFrame(table)
df_warc.to_csv('/project/0_cleaning/output_cleaning/summary-warc.csv', index=False)

100%|██████████| 605987/605987 [5:39:59<00:00, 29.71it/s]   


# Extract unique domains

Create a list of unique domains identified as ecommerce domains.

In [3]:
df_warc = pd.read_csv('/project/0_cleaning/output_cleaning/summary-warc.csv')
df_warc.head()

,content-type,file,ip,url,wapps
0,text/html; charset=utf-8,CC-MAIN-20171119004302-20171119024302-00601.wa...,80.248.213.170,https://www.boutique-waterair.fr/pages/retour....,"0 ['PrestaShop']\nName: wapps, dtype: object"
1,text/html; charset=utf-8,CC-MAIN-20171120032907-20171120052907-00562.wa...,107.154.162.186,http://www.manga-vf.fr/anime-gate-keepers-vf-s...,"1 ['OpenCart']\nName: wapps, dtype: object"
2,text/html; charset=UTF-8,CC-MAIN-20171124180349-20171124200349-00561.wa...,136.144.137.205,https://www.machemer-online.fr/tissu-en-coton-...,"2 ['Magento']\nName: wapps, dtype: object"
3,text/html; charset=utf-8,CC-MAIN-20171123050243-20171123070243-00112.wa...,165.231.2.190,http://www.bougezpourlivg.fr/Maillot-Formation...,"3 ['OpenCart']\nName: wapps, dtype: object"
4,text/html; charset=UTF-8,CC-MAIN-20171117223659-20171118003659-00491.wa...,178.32.102.200,http://www.kalifrais.fr/kalifrais-products/pot...,"4 ['Magento']\nName: wapps, dtype: object"


In [8]:
urls = []

for url in tqdm(df_warc['url']):
    url_parsed = urlparse(url)
    urls.append({
        'scheme': url_parsed.scheme,
        'domain': url_parsed.netloc,
        'url': f'{url_parsed.scheme}://{url_parsed.netloc}'
    })

df_urls = pd.DataFrame(urls)
unique_domains = np.array(df_urls['domain'].unique())
np.save('/project/0_cleaning/output_cleaning/unique_domains_ecom.npy', unique_domains)

100%|██████████| 605987/605987 [00:03<00:00, 176535.18it/s]
